In [ ]:
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
import joblib

from prophetmodel import ProphetModel
from seriesdata import SeriesDataset
from residualnn import ResidualNetwork

In [ ]:
btc = pd.read_csv('btc_preds.csv')
btc = btc.reset_index(drop=True)

macro = pd.read_csv('macro_preds.csv')
macro = macro.reset_index(drop=True)

df = pd.concat([btc, macro], axis=1)

In [ ]:
data = pd.read_csv('Preprocessed/data.csv')
test = sliced_data = data[(data['Date'] > '2023-12-31')]

df = df.dropna()
test = test.dropna()

In [ ]:
df['pred_sum'] = (0.05 * df['macro_predictions'] + 0.95 * df['btc_predictions']) 
df['pred_sum'] = df['pred_sum'] * 1.0  # Ensure floating point values
df['pred_return'] = (df['pred_sum'] - df['btc_actual']) / df['btc_actual']

df['future_btc_actual'] = df['btc_actual'].shift(-1)  
df['future_pred_sum'] = df['pred_sum'].shift(-1) 

backtest = df[['btc_actual', 'future_btc_actual', 'pred_sum', 'future_pred_sum']]
backtest.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(df['btc_actual'], df['pred_sum'])
rmse = np.sqrt(mean_squared_error(df['btc_actual'], df['pred_sum']))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

In [ ]:
import pandas as pd
import numpy as np


"""
This is a very simple backtester
This assumes we are able to buy fractional coins

The buy multiplier controls how much of our liquid cash we use to purchase BTC
The sell multiplier controls how much of our positions (btc owned) we sell
Each transaction is recorded and stored, including holds.

"""

class Backtester:
    def __init__(self, df, initial_balance=50000, buy_multiplier=0.5, sell_multiplier=0.5, output=False):
        self.df = df
        self.balance = initial_balance
        self.positions = 0  # BTC owned
        self.cash = initial_balance  
        self.buy_multiplier = buy_multiplier
        self.sell_multiplier = sell_multiplier
        self.history = []  
        self.output = output
        self.min_trade_size_btc = 0.00000001  # = 1 Satoshi

    def execute_trade(self, signal, price):
        if signal > price and self.cash > 0:
            amount_to_buy = (self.cash * self.buy_multiplier) / price  

            # Only buy if above minimum trade size
            if amount_to_buy >= self.min_trade_size_btc:
                self.positions += amount_to_buy
                self.cash -= amount_to_buy * price
                self.history.append(('Buy', price, self.cash, self.positions, self.balance))

                if self.output:
                    print(f"{price:<20} {'Buy':<20} {self.positions:<25} {self.cash:<25} {self.balance:<25}")
            else:
                self.history.append(('Hold', price, self.cash, self.positions, self.balance))

        elif signal < price and self.positions > 0:
            amount_to_sell = self.positions * self.sell_multiplier

            # Only sell if above minimum trade size
            if amount_to_sell >= self.min_trade_size_btc:
                self.cash += amount_to_sell * price  
                self.positions -= amount_to_sell
                self.history.append(('Sell', price, self.cash, self.positions, self.balance))

                if self.output:
                    print(f"{price:<20} {'Sell':<20} {self.positions:<25} {self.cash:<25} {self.balance:<25}")
            else:
                self.history.append(('Hold', price, self.cash, self.positions, self.balance))

        else:
            self.history.append(('Hold', price, self.cash, self.positions, self.balance))

    def run_backtest(self):
        if self.output:
            print(f'Initial State: Cash: {self.cash}, Positions: {self.positions}, Balance: {self.balance}')
            print()
            print(f"{'Price':<20} {'Action':<20} {'Positions':<25} {'Cash':<25} {'Balance':<25}")
            
        for idx, row in self.df.iterrows():
            price = row['btc_actual']  
            signal = row['future_pred_sum']
            self.execute_trade(signal, price)
            self.balance = self.cash + self.positions * price

        history_df = pd.DataFrame(self.history, columns=['Action', 'Price', 'Cash', 'Positions', 'Balance'])
        return history_df


In [ ]:
# Find optimal buy/sell multipliers from 0.05 to 1.0

buy_multipliers = np.arange(0.05, 1.05, 0.05)  
sell_multipliers = np.arange(0.05, 1.05, 0.05)  
results = []

for buy_multiplier in buy_multipliers:
    for sell_multiplier in sell_multipliers:

        backtester = Backtester(df, buy_multiplier=buy_multiplier, sell_multiplier=sell_multiplier)
        result = backtester.run_backtest()

        cash = result['Cash'].tail(1).iloc[0]
        balance = result['Balance'].tail(1).iloc[0]
        positions = result['Positions'].tail(1).iloc[0] 

        results.append({
            'Buy Multiplier': buy_multiplier,
            'Sell Multiplier': sell_multiplier,
            'Cash': cash,
            'Balance': balance,
            'Positions': positions
        })

results_df = pd.DataFrame(results)

In [ ]:
pd.options.display.float_format = '{:.8f}'.format

max_balance_row = results_df.loc[results_df['Balance'].idxmax()].to_frame()
min_balance_row = results_df.loc[results_df['Balance'].idxmin()].to_frame()

print("Greatest Gain:")
print(max_balance_row)

print("\nLeast Gain:")
print(min_balance_row)

In [ ]:
backtester = Backtester(df, buy_multiplier=0.30, sell_multiplier=0.15)
max_result = backtester.run_backtest()

backtester = Backtester(df, buy_multiplier=0.05, sell_multiplier=1.0)
min_result = backtester.run_backtest()

In [ ]:
# Metrics

def roi(initial, final):
    return (final - initial) / initial * 100.0

def compute_sharpe(df):
    returns = df['Balance'].pct_change().dropna()
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return
    return sharpe_ratio

def compute_sortino(df, target=0):
    returns = df['Balance'].pct_change().dropna()
    avg_return = returns.mean()
    downside = returns[returns < target]
    std_downside = downside.std()
    sortino_ratio = avg_return / std_downside
    return sortino_ratio

def max_drawdown(df):
    cumulative = df['Balance'].cummax()
    drawdown = (df['Balance'] - cumulative) / cumulative
    return drawdown.min() * 100

def calmar_ratio(df):
    returns = df['Balance'].pct_change().dropna()
    avg_return = returns.mean() * 363
    mdd = abs(max_drawdown(df)) / 100
    return avg_return / mdd


initial_balance = 50000.0
max_final = max_result.iloc[-1]['Balance']
min_final = min_result.iloc[-1]['Balance']

roi_max = roi(initial_balance, max_final)
roi_min = roi(initial_balance, min_final)

sharpe_max = compute_sharpe(max_result)
sharpe_min = compute_sharpe(min_result)


sortino_max = compute_sortino(max_result, target=0.00)
sortino_min = compute_sortino(min_result, target=0.00)

mdd_max = max_drawdown(max_result)
mdd_min = max_drawdown(min_result)

calmar_max = calmar_ratio(max_result)
calmar_min = calmar_ratio(min_result)

metrics_df = pd.DataFrame({
    'Metric': ['ROI', 'Sharpe', 'Sortino', 'MDD', 'Calmar', 'Win Rate'],
    'Max Result': [roi_max, sharpe_max, sortino_max, mdd_max, calmar_max, winrate_max],
    'Min Result': [roi_min, sharpe_min, sortino_min, mdd_min, calmar_min, winrate_min],
})

metrics_df